In [6]:
import sys
import os
sys.path.append('..')
from utils import load_data, nx2gt
from graph_tool.all import motifs, Graph
import networkx as nx
import numpy as np
from random import shuffle

# Load REDDIT-BINARY

In [3]:
reddit_graphs = load_data("REDDITBINARY", degree_as_tag=False)

loading data
# classes: 2
# maximum node tag: 1
# data: 2000


# Test counting trees

In [7]:
test_g = nx2gt(reddit_graphs[0][0].g)
# Workaround to allow parallel motifs
idx = test_g.vertex_index.copy("int")
shuffle(idx.a)
test_g = Graph(test_g, vorder=idx)

In [8]:
trees_5 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(5)]

In [9]:
len(trees_5)

3

In [11]:
%%time
motifs(g=test_g, k=5, p=1.0, motif_list=trees_5)

CPU times: user 29.2 s, sys: 29.6 ms, total: 29.3 s
Wall time: 29.2 s


([<Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2b70>,
  <Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2a20>,
  <Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2940>],
 [10622826, 491335, 8971])

In [8]:
trees_6 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(6)]

In [9]:
len(trees_6)

6

In [10]:
%%time
motifs(g=test_g, k=6, p=1.0, motif_list=[trees_6[0]])

CPU times: user 5min 33s, sys: 3.96 s, total: 5min 37s
Wall time: 5min 36s


([<Graph object, undirected, with 6 vertices and 5 edges at 0x7f57000c2da0>],
 [10471])

# Build Homomorphism profile

In [12]:
trees_2 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(2)] # <1 sec
trees_3 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(3)] # <1 sec
trees_4 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(4)] # 1 sec per tree
trees_5 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(5)] # 12 sec per tree
trees_6 = [nx2gt(f) for f in nx.generators.nonisomorphic_trees(6)] # 5 min per tree

In [13]:
all_trees = (trees_2, trees_3, trees_4, trees_5)

In [14]:
len(reddit_graphs[0])

2000

In [ ]:
%%time
# This one should run in 16-20 hrs
X = []
y = []
for g in reddit_graphs[0]:
    gt = nx2gt(g.g)
    profile = []
    for i, trees in enumerate(all_trees):
        profile.extend(motifs(g=gt, k=i+2, p=1.0, motif_list=trees)[1])
    X.append(profile)
    y.append(g.label)